In [1]:
# use the cell below to ensure correct relative imports
# you must be in the qcore directory

In [2]:
cd ../qcore

C:\Users\qcrew\Desktop\qcore\qcore


### Proto-user guide for interacting with QuantumElement and QuantumDevice objects in qcore

Motivation: you want a way to store the properties of your cQED samples for easy getting/setting.

1. A QuantumElement encapsulates a qubit, resonator e.t.c 
2. A QuantumDevice encapsulates a cQED package composed of quantum elements (qubits, resonators e.t.c)
3. In qcore, both object types are of type MetaInstrument, so you can initialize them with a variable number of parameters, and once initialized, their parameters become their attributes - more on this below!
4. Moreover, both object types can be saved into yaml files - more on this below too!

In [3]:
# import cqed components
from instruments import QuantumElement, QuantumDevice

### Creating a qubit and a resonator

In [4]:
# we can initialize a QuantumElement with a 'name' and any number of other parameters
# let's first create a dictionary of these parameters

# in this example, we define the minimal parameters needed for the OPX to interact with a qubit and a resonator

qubit_parameters = {
    'name': 'qubit',
    'lo_freq': 5e9,
    'int_freq': 50e6,
    'ports': {'I': 1, 'Q': 2}, # analog output ports on the OPX
    'operations': {}
}

rr_parameters = {
    'name': 'rr',
    'lo_freq': 8e9,
    'int_freq': -50e6,
    'ports': {'I': 3, 'Q': 4, 'out': 1}, # 'out' refers to analog input port on the OPX
    'operations': {},
    'time_of_flight': 824,
    'smearing': 0
}

In [5]:
# let's now add some operations to the qubit and resonator from the pulse library
# we import 3 default Pulse objects
from utils.pulselib import DEFAULT_CW_PULSE, DEFAULT_READOUT_PULSE, DEFAULT_GAUSSIAN_PULSE

# here, 'CW' is the name of the operation and DEFAULT_CW_PULSE is the Pulse object associated with the operation
qubit_parameters['operations']['CW'] = DEFAULT_CW_PULSE
qubit_parameters['operations']['gaussian'] = DEFAULT_GAUSSIAN_PULSE

# similarly, let's add a CW and a readout pulse to our resonator operations
rr_parameters['operations']['CW'] = DEFAULT_CW_PULSE
rr_parameters['operations']['readout'] = DEFAULT_READOUT_PULSE

In [6]:
# let's create our qubit and resonator objects!!!
qubit = QuantumElement(**qubit_parameters) # ** means we are unpacking the dictionary
rr = QuantumElement(**rr_parameters)

### Interacting with the qubit and resonator

In [7]:
# as QuantumElement objects are MetaInstruments, every parameter they have is also their attribute!
# you can first call .parameters to see what parameters they have

In [8]:
qubit.parameters

{'lo_freq': 5000000000.0,
 'int_freq': 50000000.0,
 'ports': {'I': 1, 'Q': 2},
 'operations': {'CW': <utils.pulselib.Pulse at 0x1c3c2c05f70>,
  'gaussian': <utils.pulselib.Pulse at 0x1c3c2c330a0>}}

In [9]:
rr.parameters

{'lo_freq': 8000000000.0,
 'int_freq': -50000000.0,
 'ports': {'I': 3, 'Q': 4, 'out': 1},
 'operations': {'CW': <utils.pulselib.Pulse at 0x1c3c2c05f70>,
  'readout': <utils.pulselib.MeasurementPulse at 0x1c3c2c05fa0>},
 'time_of_flight': 824,
 'smearing': 0}

In [10]:
# now, let's access the qubit's lo_freq
qubit.lo_freq

# and so on...
# you might think this object is a little boring, you can't interact with it much, and you'll be right
# this class currently exists to store ALL the data about your quantum element!

5000000000.0

### Packaging the quantum elements in QuantumDevice

In [11]:
# but what if you wanted to store the data of your entire DEVICE ?!? We have QuantumDevice just for that!!
quantum_elements = {
    'qubit': qubit, # key is the name, value is the QuantumElement object
    'rr': rr
}

device = QuantumDevice(name='device_A', **quantum_elements)

In [12]:
# just like QuantumElement, QuantumDevice is also a MetaInstrument, which means you can call its parameters as its attributes!
# you'll get the current parameter values of all elements that are part of this device
device.parameters

{'qubit': {'lo_freq': 5000000000.0,
  'int_freq': 50000000.0,
  'ports': {'I': 1, 'Q': 2},
  'operations': {'CW': <utils.pulselib.Pulse at 0x1c3c2c05f70>,
   'gaussian': <utils.pulselib.Pulse at 0x1c3c2c330a0>}},
 'rr': {'lo_freq': 8000000000.0,
  'int_freq': -50000000.0,
  'ports': {'I': 3, 'Q': 4, 'out': 1},
  'operations': {'CW': <utils.pulselib.Pulse at 0x1c3c2c05f70>,
   'readout': <utils.pulselib.MeasurementPulse at 0x1c3c2c05fa0>},
  'time_of_flight': 824,
  'smearing': 0}}

In [13]:
# to access the qubit, you use
device.qubit

# to access the resonator, you use
device.rr

### Saving QuantumDevice to yaml file

In [14]:
# we can of course save the individual QuantumElement, but it makes more sense to save the whole device, so let's do that

# first, we import Python's pathlib, which makes dealing with paths REALLY STRAIGHTFORWARD
from pathlib import Path

In [15]:
# I want my yaml file to be saved with a given name in the 'tutorials' folder, so I do
yaml_file_name = 'example_device.yaml'
yaml_file_path = Path.cwd().parent / 'tutorials' / yaml_file_name

In [16]:
# i now call save() on the device and pass it the yaml_file_path (which is a Path object)
device.save(yaml_file_path)

# you can now use a Stage object to load this device in future experiments - more on this in the stage tutorial!